In [39]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import multilabel_confusion_matrix # For separate confusion matrix for each class
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, auc

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")
import json
import pickle

## Problem satatement

In [ ]:
predict Loan eligiltity 

## Data Gathering

In [5]:
df = pd.read_csv("loan_data.csv")
df

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,0,all_other,0.1461,344.76,12.180755,10.39,672,10474.000000,215372,82.1,2,0,0,1
9574,0,all_other,0.1253,257.70,11.141862,0.21,722,4380.000000,184,1.1,5,0,0,1
9575,0,debt_consolidation,0.1071,97.81,10.596635,13.09,687,3450.041667,10036,82.9,8,0,0,1
9576,0,home_improvement,0.1600,351.58,10.819778,19.18,692,1800.000000,0,3.2,5,0,0,1


## EDA

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit.policy      9578 non-null   int64  
 1   purpose            9578 non-null   object 
 2   int.rate           9578 non-null   float64
 3   installment        9578 non-null   float64
 4   log.annual.inc     9578 non-null   float64
 5   dti                9578 non-null   float64
 6   fico               9578 non-null   int64  
 7   days.with.cr.line  9578 non-null   float64
 8   revol.bal          9578 non-null   int64  
 9   revol.util         9578 non-null   float64
 10  inq.last.6mths     9578 non-null   int64  
 11  delinq.2yrs        9578 non-null   int64  
 12  pub.rec            9578 non-null   int64  
 13  not.fully.paid     9578 non-null   int64  
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB


In [7]:
df.isna().sum()

credit.policy        0
purpose              0
int.rate             0
installment          0
log.annual.inc       0
dti                  0
fico                 0
days.with.cr.line    0
revol.bal            0
revol.util           0
inq.last.6mths       0
delinq.2yrs          0
pub.rec              0
not.fully.paid       0
dtype: int64

In [8]:
df["purpose"].isna().sum()

0

In [9]:
# it is object so we have convert by label encoding

df["purpose"].value_counts().to_dict()

{'debt_consolidation': 3957,
 'all_other': 2331,
 'credit_card': 1262,
 'home_improvement': 629,
 'small_business': 619,
 'major_purchase': 437,
 'educational': 343}

In [10]:
df["purpose"].replace({'debt_consolidation': 1,
 'all_other': 2,
 'credit_card': 3,
 'home_improvement': 4,
 'small_business': 5,
 'major_purchase': 6,
 'educational': 0},inplace = True)

In [34]:
purpose_values = {'debt_consolidation': 1,
 'all_other': 2,
 'credit_card': 3,
 'home_improvement': 4,
 'small_business': 5,
 'major_purchase': 6,
 'educational': 0}

In [11]:
df

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,1,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,3,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,1,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,1,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,3,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,0,2,0.1461,344.76,12.180755,10.39,672,10474.000000,215372,82.1,2,0,0,1
9574,0,2,0.1253,257.70,11.141862,0.21,722,4380.000000,184,1.1,5,0,0,1
9575,0,1,0.1071,97.81,10.596635,13.09,687,3450.041667,10036,82.9,8,0,0,1
9576,0,4,0.1600,351.58,10.819778,19.18,692,1800.000000,0,3.2,5,0,0,1


In [12]:
x = df.drop("credit.policy",axis = 1)
y = df["credit.policy"]

In [13]:
x

,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,3,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,3,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,2,0.1461,344.76,12.180755,10.39,672,10474.000000,215372,82.1,2,0,0,1
9574,2,0.1253,257.70,11.141862,0.21,722,4380.000000,184,1.1,5,0,0,1
9575,1,0.1071,97.81,10.596635,13.09,687,3450.041667,10036,82.9,8,0,0,1
9576,4,0.1600,351.58,10.819778,19.18,692,1800.000000,0,3.2,5,0,0,1


In [14]:
y

0       1
1       1
2       1
3       1
4       1
       ..
9573    0
9574    0
9575    0
9576    0
9577    0
Name: credit.policy, Length: 9578, dtype: int64

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=30,stratify=y) 

In [16]:
x_train

,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
5627,2,0.1357,509.53,10.703244,4.88,717,4589.958333,6588,79.4,1,0,0,0
7722,1,0.1387,231.98,11.512925,14.64,657,3498.000000,39411,97.8,1,0,0,0
5645,2,0.1114,590.47,11.630709,0.99,772,4574.041667,0,0.0,0,0,0,0
3122,1,0.0963,104.31,8.987197,13.80,722,4380.000000,5136,34.9,2,0,0,1
62,2,0.1028,113.39,9.903488,1.50,682,1410.000000,1882,32.4,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,3,0.0894,476.58,11.608236,7.07,797,6510.958333,7586,52.7,1,0,0,0
8424,3,0.1861,759.27,11.512925,18.89,652,2460.041667,66900,94.0,7,0,0,0
8758,2,0.1538,174.26,10.778956,15.65,652,4020.000000,901,42.9,1,0,1,0
6902,5,0.1809,202.70,9.798127,9.80,682,1230.041667,0,0.0,0,0,0,0


In [17]:
y_train

5627    1
7722    0
5645    1
3122    1
62      1
       ..
17      1
8424    0
8758    0
6902    1
6141    1
Name: credit.policy, Length: 7183, dtype: int64

In [18]:
columns = x.columns
columns

Index(['purpose', 'int.rate', 'installment', 'log.annual.inc', 'dti', 'fico',
       'days.with.cr.line', 'revol.bal', 'revol.util', 'inq.last.6mths',
       'delinq.2yrs', 'pub.rec', 'not.fully.paid'],
      dtype='object')

## Model Training

In [19]:
model  = LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [20]:
y_pred = model.predict(x_test)

In [24]:
## Testing data
cnf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n",cnf_matrix)

print("--"*50)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score:",accuracy)
print("--"*50)

clf_report = classification_report(y_test, y_pred)
print("Classification report:\n",clf_report)

Confusion Matrix:
 [[ 198  269]
 [  37 1891]]
----------------------------------------------------------------------------------------------------
Accuracy Score: 0.8722338204592902
----------------------------------------------------------------------------------------------------
Classification report:
               precision    recall  f1-score   support

           0       0.84      0.42      0.56       467
           1       0.88      0.98      0.93      1928

    accuracy                           0.87      2395
   macro avg       0.86      0.70      0.74      2395
weighted avg       0.87      0.87      0.85      2395



In [25]:
y_train_pred = model.predict(x_train)

In [26]:
## Training data
cnf_matrix = confusion_matrix(y_train, y_train_pred)
print("Confusion Matrix:\n",cnf_matrix)

print("--"*50)
accuracy = accuracy_score(y_train, y_train_pred)
print("Accuracy Score:",accuracy)
print("--"*50)

clf_report = classification_report(y_train, y_train_pred)

Confusion Matrix:
 [[ 583  818]
 [ 121 5661]]
----------------------------------------------------------------------------------------------------
Accuracy Score: 0.8692746763190867
----------------------------------------------------------------------------------------------------


In [27]:
# prediction probability
y_pred_proba = model.predict_proba(x_test)
y_pred_proba

array([[0.1192578 , 0.8807422 ],
       [0.24031456, 0.75968544],
       [0.43910993, 0.56089007],
       ...,
       [0.05835259, 0.94164741],
       [0.24493519, 0.75506481],
       [0.44621932, 0.55378068]])

In [28]:
y_train_pred = model.predict(x_train)
y_train_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [29]:
x.head(1).T

,0
purpose,1.000000
int.rate,0.118900
installment,829.100000
log.annual.inc,11.350407
dti,19.480000
fico,737.000000
days.with.cr.line,5639.958333
revol.bal,28854.000000
revol.util,52.100000
inq.last.6mths,0.000000


In [31]:
x.tail(1).T

,9577
purpose,1.000000
int.rate,0.139200
installment,853.430000
log.annual.inc,11.264464
dti,16.280000
fico,732.000000
days.with.cr.line,4740.000000
revol.bal,37879.000000
revol.util,57.000000
inq.last.6mths,6.000000


In [32]:
purpose = "debt_consolidation"
int_rate = 0.118900
installment = 853.430000
log_annual_inc = 11.350407
dti = 16.280000
fico = 732.000000
days_with_cr_line = 4740.000000
revol_bal = 37879.000000
revol_util = 57.000000
inq_last_6mths = 6.000000
delinq_2yrs = 0.000000
pub_rec = 0.000000
not_fully_paid = 1.00000

In [35]:
array = np.zeros(len(columns))
array[0] = purpose_values[purpose]
array[1] = int_rate
array[2] = installment
array[3] =log_annual_inc
array[4] = dti
array[5] = fico
array[6] = days_with_cr_line
array[7] = revol_bal
array[8] = revol_util
array[9] = inq_last_6mths
array[10] = delinq_2yrs
array[11] = pub_rec
array[12]= not_fully_paid
array

array([1.0000000e+00, 1.1890000e-01, 8.5343000e+02, 1.1350407e+01,
       1.6280000e+01, 7.3200000e+02, 4.7400000e+03, 3.7879000e+04,
       5.7000000e+01, 6.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.0000000e+00])

In [36]:
prediction = model.predict([array])[0]
prediction

0

In [42]:
dictionary = {"purpose_values" : purpose_values,"columns" : list(columns)}

In [43]:
purpose = "home_improvement"
int_rate = 0.118900
installment = 829.100000
log_annual_inc = 11.350407
dti = 19.480000
fico = 737.000000
days_with_cr_line = 5639.958333
revol_bal = 28854.000000
revol_util = 52.100000
inq_last_6mths = 0.000000
delinq_2yrs = 0.000000
pub_rec = 0.000000
not_fully_paid = 0.00000

In [44]:
with open("loan_data.pkl","wb") as f:
    pickle.dump(model,f)
 

In [45]:
with open("loan.json","w") as f:
    json.dump(dictionary,f)